# Method 1: Use mysql-connector-python

In [4]:
## Install lib
!pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [16]:
import mysql.connector
import pandas as pd


# Database connection parameters
config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'port': 3306,
    'database': 'hr',
    'raise_on_warnings': True
}

try:
    # Establishing a connection to the database
    conn = mysql.connector.connect(**config)
    print("Connection established")

    # Create a cursor object
    cursor = conn.cursor()

    # Your SQL query (for example, selecting all rows from a table)
    query = "SELECT * FROM departments"

    # Executing the query
    cursor.execute(query)

    # Fetching the rows from the query result
    rows = cursor.fetchall()
    for row in rows:
        print(row)

except mysql.connector.Error as err:
    print(f"Error: {err}")

# finally:
#     if conn.is_connected():
#         cursor.close()
#         conn.close()
#         print("Connection closed")



Connection established
('2', 'Architect Group', '30001', 'L0001')
('5', 'Software Group', '30002', 'L0002')
('7', 'Design Team', '30003', 'L0003')


In [17]:
!pip install sqlalchemy pymysql

Defaulting to user installation because normal site-packages is not writeable


In [21]:
df = pd.read_sql(query, conn)
df.head(5)

C:\Users\giang\AppData\Local\Temp\ipykernel_17876\714362292.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
0           2  Architect Group      30001  L0001
1           5   Software Group      30002  L0002
2           7      Design Team      30003  L0003

In [22]:
df.describe()

DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
count            3                3          3      3
unique           3                3          3      3
top              2  Architect Group      30001  L0001
freq             1                1          1      1

# Method 2: Use sqlalchemy

In [23]:
!pip install sqlalchemy pymysql


Defaulting to user installation because normal site-packages is not writeable


In [30]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Database connection parameters
user= 'root'
password = ''
host = 'localhost'
port = 3306
database = 'hr'
table_name = 'departments'

###### Method 1
# Create a connection string (URL)
connection_string = f'mysql+pymysql://{user}:{password}@{host}/{database}'
# Create an engine
engine = create_engine(connection_string)

####### Method 2
# Define the connection parameters and the engine
# connection_string = 'mysql+pymysql://root:@localhost:3306/hr'
# engine = create_engine(connection_string)

# Read the table into a DataFrame
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, engine)

print(df)


  DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
0          13       Researcher      30006  L0007
1           2  Architect Group      30001  L0001
2           5   Software Group      30002  L0002
3           7      Design Team      30003  L0003


In [31]:
##  show all the tables in a specific database

# Connect to the database and retrieve the list of tables
with engine.connect() as connection:
    result = connection.execute(text("SHOW TABLES"))
    tables = [row[0] for row in result]
    print("Tables in the 'hr' database:", tables)


Tables in the 'hr' database: ['departments', 'employees', 'job_history', 'jobs', 'locations']


## Insert

In [32]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Create engine (modify with actual database credentials)
#engine = create_engine("mysql+pymysql://root:@localhost:3306/hr")
engine = create_engine(connection_string)


# Define new department data
new_department_data = {
    'DEPT_ID_DEP': 13, 
    'DEP_NAME': 'Researcher', 
    'MANAGER_ID': 30006, 
    'LOC_ID': 'L0007'
}

# SQL Insert Query using `text()`
query = text("""
    INSERT INTO departments (DEPT_ID_DEP, DEP_NAME, MANAGER_ID, LOC_ID) 
    VALUES (:DEPT_ID_DEP, :DEP_NAME, :MANAGER_ID, :LOC_ID)
""")

# Execute the query correctly
try:
    with engine.connect() as connection:
        result = connection.execute(query, new_department_data)  # ✅ Pass as a dictionary
        connection.commit()  # ✅ Ensure data is committed (if autocommit is disabled)
        print(f"Inserted {result.rowcount} records into the departments table.")
except SQLAlchemyError as e:
    print(f"An error occurred: {e}")
finally:
    engine.dispose()


An error occurred: (pymysql.err.IntegrityError) (1062, "Duplicate entry '13' for key 'PRIMARY'")
[SQL: 
    INSERT INTO departments (DEPT_ID_DEP, DEP_NAME, MANAGER_ID, LOC_ID) 
    VALUES (%(DEPT_ID_DEP)s, %(DEP_NAME)s, %(MANAGER_ID)s, %(LOC_ID)s)
]
[parameters: {'DEPT_ID_DEP': 13, 'DEP_NAME': 'Researcher', 'MANAGER_ID': 30006, 'LOC_ID': 'L0007'}]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [33]:
# Read the table into a DataFrame - check insert
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, engine)

print(df)

  DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
0          13       Researcher      30006  L0007
1           2  Architect Group      30001  L0001
2           5   Software Group      30002  L0002
3           7      Design Team      30003  L0003


## Update

In [34]:
# Update data
# Create engine (modify with actual database credentials)
engine = create_engine(connection_string)

# Update data
update_data = {
    'DEPT_ID_DEP': 12,
    'NEW_DEP_NAME': 'Business Analytics',
    'NEW_MANAGER_ID': 30010
}

# SQL Update Query
update_query = text("""
    UPDATE departments
    SET DEP_NAME = :NEW_DEP_NAME, MANAGER_ID = :NEW_MANAGER_ID
    WHERE DEPT_ID_DEP = :DEPT_ID_DEP
""")

# Execute the update
try:
    with engine.connect() as connection:
        result = connection.execute(update_query, update_data)  # ✅ Pass parameters as a dictionary
        connection.commit()  # ✅ Ensure the transaction is committed
        print(f"Updated {result.rowcount} records in the departments table.")
except SQLAlchemyError as e:
    print(f"An error occurred: {e}")
finally:
    engine.dispose()

Updated 0 records in the departments table.


In [35]:
# Read the table into a DataFrame - check updated
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, engine)

print(df)

  DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
0          13       Researcher      30006  L0007
1           2  Architect Group      30001  L0001
2           5   Software Group      30002  L0002
3           7      Design Team      30003  L0003


## Delete

In [36]:
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError


# Create engine
engine = create_engine(connection_string)

# Data to specify which department to delete
delete_data = {
    'DEPT_ID_DEP': 12
}

# SQL Delete Query
delete_query = text("""
    DELETE FROM departments
    WHERE DEPT_ID_DEP = :DEPT_ID_DEP
""")

# Execute the delete operation
try:
    with engine.connect() as connection:
        result = connection.execute(delete_query, delete_data)  # ✅ Pass as a dictionary
        connection.commit()  # ✅ Commit the transaction to apply the delete
        print(f"Deleted {result.rowcount} records from the departments table.")
except SQLAlchemyError as e:
    print(f"An error occurred: {e}")
finally:
    engine.dispose()


Deleted 0 records from the departments table.


In [37]:
# Read the table into a DataFrame - check delete
query = f"SELECT * FROM {table_name}"
df = pd.read_sql(query, engine)

print(df)

  DEPT_ID_DEP         DEP_NAME MANAGER_ID LOC_ID
0          13       Researcher      30006  L0007
1           2  Architect Group      30001  L0001
2           5   Software Group      30002  L0002
3           7      Design Team      30003  L0003


## Assignment 3: Database Manipulation with Python
Objective: Practice database manipulation using Python to perform insert, update, and delete operations on the 'employees' table in the HR database.

Background: You have already learned how to connect to a MySQL database (we use XAMPP) and perform data manipulation language (DML) operations on the 'departments' table. Now, you will apply these skills to the 'employees' table to further your understanding of database interactions using Python. 


In [46]:
# Define new employee data
new_employee = {
    'EMP_ID': 'E1015',  
    'F_NAME': 'Alice',
    'L_NAME': 'Johnson',
    'SSN': '456789123',
    'B_DATE': '1995-03-15',
    'SEX': 'F',
    'ADDRESS': '123 Green St, NY',
    'JOB_ID': 700,
    'SALARY': 75000,
    'MANAGER_ID': 30005,
    'DEP_ID': 5
}

# SQL Insert Query
insert_query = text("""
    INSERT INTO employees (EMP_ID, F_NAME, L_NAME, SSN, B_DATE, SEX, ADDRESS, JOB_ID, SALARY, MANAGER_ID, DEP_ID) 
    VALUES (:EMP_ID, :F_NAME, :L_NAME, :SSN, :B_DATE, :SEX, :ADDRESS, :JOB_ID, :SALARY, :MANAGER_ID, :DEP_ID)
""")

# Execute Insert
try:
    with engine.connect() as connection:
        with connection.begin():  
            result = connection.execute(insert_query, new_employee)
            print(f"✅ Inserted {result.rowcount} record(s) into employees table.")
except SQLAlchemyError as e:
    print(f"❌ An error occurred: {e}")


✅ Inserted 1 record(s) into employees table.


In [47]:
query = "SELECT * FROM employees"
df = pd.read_sql(query, engine)

print(df)


   EMP_ID   F_NAME   L_NAME        SSN      B_DATE SEX  \
0   E1001     John   Thomas     123456  1976-09-01   M   
1   E1002    Alice    James     123457  1972-07-31   F   
2   E1003    Steve    Wells     123458  1980-10-08   M   
3   E1004  Santosh    Kumar     123459  1985-07-20   M   
4   E1005    Ahmed  Hussain     123410  1981-04-01   M   
5   E1006    Nancy    Allen     123411  1978-06-02   F   
6   E1007     Mary   Thomas     123412  1975-05-05   F   
7   E1008  Bharath    Gupta     123413  1985-06-05   M   
8   E1009   Andrea    Jones     123414  1990-09-07   F   
9   E1010      Ann    Jacob     123415  1982-03-30   F   
10  E1015    Alice  Johnson  456789123  1995-03-15   F   

                         ADDRESS JOB_ID    SALARY MANAGER_ID DEP_ID  
0          5631 Rice, OakPark,IL    100  100000.0      30001      2  
1         980 Berry ln, Elgin,IL    200   80000.0      30002      5  
2           291 Springs, Gary,IL    300   50000.0      30002      5  
3       511 Aurora Av, 

In [48]:
# Define Update Data
update_data = {
    'EMP_ID': 'E1015',  
    'NEW_SALARY': 85000
}

# SQL Update Query
update_query = text("""
    UPDATE employees
    SET SALARY = :NEW_SALARY
    WHERE EMP_ID = :EMP_ID
""")

# Execute Update
try:
    with engine.connect() as connection:
        with connection.begin():  
            result = connection.execute(update_query, update_data)
            print(f"✅ Updated {result.rowcount} record(s) in employees table.")
except SQLAlchemyError as e:
    print(f"❌ An error occurred: {e}")


✅ Updated 1 record(s) in employees table.


In [49]:
query = "SELECT * FROM employees"
df = pd.read_sql(query, engine)

print(df)


   EMP_ID   F_NAME   L_NAME        SSN      B_DATE SEX  \
0   E1001     John   Thomas     123456  1976-09-01   M   
1   E1002    Alice    James     123457  1972-07-31   F   
2   E1003    Steve    Wells     123458  1980-10-08   M   
3   E1004  Santosh    Kumar     123459  1985-07-20   M   
4   E1005    Ahmed  Hussain     123410  1981-04-01   M   
5   E1006    Nancy    Allen     123411  1978-06-02   F   
6   E1007     Mary   Thomas     123412  1975-05-05   F   
7   E1008  Bharath    Gupta     123413  1985-06-05   M   
8   E1009   Andrea    Jones     123414  1990-09-07   F   
9   E1010      Ann    Jacob     123415  1982-03-30   F   
10  E1015    Alice  Johnson  456789123  1995-03-15   F   

                         ADDRESS JOB_ID    SALARY MANAGER_ID DEP_ID  
0          5631 Rice, OakPark,IL    100  100000.0      30001      2  
1         980 Berry ln, Elgin,IL    200   80000.0      30002      5  
2           291 Springs, Gary,IL    300   50000.0      30002      5  
3       511 Aurora Av, 

In [50]:
# Define Delete Data
delete_data = {
    'EMP_ID': 'E1015'
}

# SQL Delete Query
delete_query = text("""
    DELETE FROM employees
    WHERE EMP_ID = :EMP_ID
""")

# Execute Delete
try:
    with engine.connect() as connection:
        with connection.begin():  
            result = connection.execute(delete_query, delete_data)
            print(f"✅ Deleted {result.rowcount} record(s) from employees table.")
except SQLAlchemyError as e:
    print(f"❌ An error occurred: {e}")


✅ Deleted 1 record(s) from employees table.


In [51]:
query = "SELECT * FROM employees"
df = pd.read_sql(query, engine)

print(df)


  EMP_ID   F_NAME   L_NAME     SSN      B_DATE SEX  \
0  E1001     John   Thomas  123456  1976-09-01   M   
1  E1002    Alice    James  123457  1972-07-31   F   
2  E1003    Steve    Wells  123458  1980-10-08   M   
3  E1004  Santosh    Kumar  123459  1985-07-20   M   
4  E1005    Ahmed  Hussain  123410  1981-04-01   M   
5  E1006    Nancy    Allen  123411  1978-06-02   F   
6  E1007     Mary   Thomas  123412  1975-05-05   F   
7  E1008  Bharath    Gupta  123413  1985-06-05   M   
8  E1009   Andrea    Jones  123414  1990-09-07   F   
9  E1010      Ann    Jacob  123415  1982-03-30   F   

                        ADDRESS JOB_ID    SALARY MANAGER_ID DEP_ID  
0         5631 Rice, OakPark,IL    100  100000.0      30001      2  
1        980 Berry ln, Elgin,IL    200   80000.0      30002      5  
2          291 Springs, Gary,IL    300   50000.0      30002      5  
3      511 Aurora Av, Aurora,IL    400   60000.0      30004      5  
4       216 Oak Tree, Geneva,IL    500   70000.0      30001 